<h1>About Dataset</h1>
<h2>Dataset Overview</h2>
<p>The <strong>Spending Patterns Dataset</strong> provides a synthetic representation of consumer spending behavior across various categories. This dataset is ideal for exploratory data analysis, statistical modeling, and machine learning applications related to financial forecasting, customer segmentation, or consumer behavior analysis.</p>
<h2>Dataset Features</h2>
<p>The dataset contains 10,000 transactions for 200 unique customers. Each transaction is associated with detailed information, including category, item, quantity, price, payment method, and transaction date.</p>
Data Source from Kaggle <br/>
<a href="https://www.kaggle.com/datasets/ahmedmohamed2003/spending-habits">Spending Habits</a>

In [14]:
# Import Library
import pandas as pd

#### 1. Data Collecting and Data Cleaning

In [3]:
df = pd.read_csv("spending_patterns_detailed.csv")
df.head()

,Customer ID,Category,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,CUST_0159,Groceries,Milk,1,1.28,1.28,Debit Card,Mobile App,2024-11-13
1,CUST_0017,Friend Activities,Dinner with Friends,2,74.69,149.39,Debit Card,In-store,2023-05-04
2,CUST_0094,Housing and Utilities,Water Bill,1,76.06,76.06,Digital Wallet,Mobile App,2023-12-16
3,CUST_0162,Fitness,Yoga Class,5,11.24,56.18,Cash,In-store,2024-01-28
4,CUST_0129,Gifts,Flowers,3,43.71,131.12,Debit Card,In-store,2023-07-28


In [10]:
df.shape

(10000, 9)

#### 2. Data Preprocessing

In [4]:
df.isnull().sum()

Customer ID         0
Category            0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [8]:
duplicate = df[df.duplicated()]
print(duplicate)

Empty DataFrame
Columns: [Customer ID, Category, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []


In [12]:
df.dtypes

Customer ID          object
Category             object
Item                 object
Quantity              int64
Price Per Unit      float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
dtype: object

In [13]:
df['Payment Method'].unique()

array(['Debit Card', 'Digital Wallet', 'Cash', 'Credit Card'],
      dtype=object)